# evaluasi code (referensi code Bu Ade)

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ql32zp21
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ql32zp21
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=9b231d3fb6fac639710c25fad655804dc53b5331931405cd61cbd994dbbe43a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-66_4kh5r/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
!pip install torch torchvision
!pip install transformers

In [ ]:
import torch
from transformers import CLIPModel, CLIPProcessor
from tqdm.auto import tqdm
import clip
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
model, preprocess = clip.load("ViT-B/32")

100%|███████████████████████████████████████| 338M/338M [00:09<00:00, 38.8MiB/s]


In [ ]:
transform = transforms.Compose([  # Ubah ukuran gambar
    transforms.ToTensor()       # Ubah ke tensor
])

In [ ]:
df = "/content/drive/MyDrive/DATA PENGPOL/data/train normalisasi/Accident"
dataset = datasets.ImageFolder(root=df, transform = transform)
# Buat DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
def zeroshot_classifier(classnames, templates):
    """
    This function generates the zeroshot weights for the given class names and templates.
    """
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):  # Using tqdm for progress bar
            texts = [template.format(classname) for template in templates]
            texts = clip.tokenize(texts)
            class_embeddings = model.encode_text(texts)
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1)
    return zeroshot_weights

imagenet_classes = [
    "A car accident on a road",
    "A car accident involving multiple vehicles",
    "A motorcycle accident involving multiple vehicles",
    "A damaged vehicle after a collision",
    "An overturned car on the highway",
    "Emergency services responding to a road accident",
    "A photo of a traffic jam caused by an accident",
]
imagenet_templates = ["{}"]

zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [ ]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images
        target = target
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/22 [00:00<?, ?it/s]

<ipython-input-23-e5b7f54f8676>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]


Top-1 accuracy: 32.00
Top-5 accuracy: 95.86
